# We have modified the code file to correct a few errors.

### Self Consistency Prompting

Self consistency is a smarter way for LLMs to solve complex problems by exploring many different reasoning paths and then choosing the most common solution, significantly improving their problem-solving abilities. Through extensive testing on various challenging questions, the method proved to be highly effective, marking a big step forward in how LLMs understand and tackle complex tasks. You can more details about this prompting technique here - https://visualsummary.substack.com/p/self-consistency-prompting

### Loading libraries

In [1]:
# os: Operating system interactions (file/directory manipulation).
import os

# operator.itemgetter: Extracts items from sequences.
from operator import itemgetter

# langchain_openai.ChatOpenAI: Integration with OpenAI's chat models for text generation.
from langchain_openai import ChatOpenAI

# langchain.prompts.PromptTemplate: Predefined templates for generating prompts.
from langchain.prompts import PromptTemplate

# langchain.schema.StrOutputParser: Parses string outputs into structured data.
from langchain.schema import StrOutputParser

# langchain.schema.runnable.RunnablePassthrough: Wraps runnable objects for easier execution.
from langchain.schema.runnable import RunnablePassthrough

# IPython.display: Displays output in Jupyter notebooks, including Markdown.
from IPython.display import display, Markdown

### Setting OpenAI API Key

In [2]:
os.environ["OPENAI_API_KEY"] = open('key.txt','r').read()

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv('/.env')

True

### Prompting

In [4]:
# Define a question about apple quantities and usage.
question = """The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""

# Provide context for answering questions, emphasizing detailed math and reasoning.
context = """Answer questions showing the full math and reasoning. Follow the pattern in the example."""

# Example problem-solving scenario involving arithmetic calculations.
one_shot_example = """Example 
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he 
have now? 

A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: """

In [10]:
# Generate a structured response using PromptTemplate, ChatOpenAI, and StrOutputParser.
answer_1 = (
    
    # Start with a base response template followed by "A: ".
    PromptTemplate.from_template(context + one_shot_example + " {input}")
    
    # Use ChatOpenAI with temperature set to 0 for deterministic output.
    | ChatOpenAI(temperature=0)
    
    # Parse the generated text into structured data.
    | StrOutputParser()
)

In [11]:
# Generate a structured response using PromptTemplate, ChatOpenAI, and StrOutputParser.
answer_2 = (
    
    # Start with a base response template followed by "A: ".
    PromptTemplate.from_template(context + one_shot_example + " {input}")
    
    # Use ChatOpenAI with temperature set to 0 for deterministic output.
    | ChatOpenAI(temperature=0.1)
    
    # Parse the generated text into structured data.
    | StrOutputParser()
)

In [12]:
# Generate a structured response using PromptTemplate, ChatOpenAI, and StrOutputParser.
answer_3 = (
    
    # Start with a base response template followed by "A: ".
    PromptTemplate.from_template(context + one_shot_example + " {input}")
    
    # Use ChatOpenAI with temperature set to 0 for deterministic output.
    | ChatOpenAI(temperature=0.7)
    
    # Parse the generated text into structured data.
    | StrOutputParser()
)

In [13]:
# Final responder setup for formatting and displaying results in markdown.
final_responder = (
    
    # Template for outputting results in markdown format.
    PromptTemplate.from_template(
        """Output all the final results in this markdown format: Result 1: {results_1} \n Result 2:{results_2} \n Result 3: 
        {results_3}"""
    )
    
    # Process the template through ChatOpenAI for text generation.
    | ChatOpenAI()
    
    # Parse the generated text into structured data.
    | StrOutputParser()
)

In [17]:
# Chain together components for a comprehensive response process.
chain = (
    
    # Map each answer to specific variables.
     {
        "results_1": answer_1,
        "results_2": answer_2,
        "results_3": answer_3,
    }
    
    # Finalize the response with formatting and display.
    | final_responder
)

In [15]:
# Invoke the response chain with a given input question.
answers = chain.invoke({"input": question})

# Display the generated answers in Markdown format.
display(Markdown(answers))

- Result 1: 
    A: The cafeteria started with 23 apples. They used 20 apples for lunch, leaving 3 apples. They then bought 6 more apples. 3 + 6 = 9. The cafeteria now has 9 apples. 

- Result 2: 
    A: The cafeteria started with 23 apples. They used 20 apples for lunch, leaving 23 - 20 = 3 apples. Then they bought 6 more apples, making a total of 3 + 6 = 9 apples. The answer is 9. 

- Result 3: 
    A: The cafeteria started with 23 apples. They used 20 for lunch, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they have 3 + 6 = 9 apples now. The answer is 9.

In [16]:
answer_1

PromptTemplate(input_variables=['input'], template='Answer questions showing the full math and reasoning. Follow the pattern in the example.Example \nQ: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he \nhave now? \n\nA: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.\n\nQ:  {input}')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7e8464c50750>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7e8464c3db90>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()